In [ ]:
%matplotlib widget

In [ ]:
from pathlib import Path
import numpy as np
from bouter import EmbeddedExperiment
import pandas as pd
from tqdm import tqdm
import flammkuchen as fl

from matplotlib import pyplot as plt
import seaborn as sns
sns.set(palette="deep", style="ticks")

from scipy.signal import detrend

from bouter.utilities import crop, reliability
from utilities import stimulus_df_from_exp

In [ ]:
master_path = Path("/Volumes/Shared/experiments/E0070_receptive_field/v04_flashing_rad_simple")


path_list = [f.parent for f in master_path.glob("*/data_from_suite2p_unfiltered.h5")]
path_list

In [ ]:
# Check genotype consistency; odd numbers should all be controls:
for path in path_list:
    exp = EmbeddedExperiment(path)
    if ((int(path.name.split("_f")[1]) % 2) != 0) != (exp["general"]["animal"]["genotype"]== "Huc:H2B-GCaMP6s"):
        print("Check metadata of fish ", path.name)

In [ ]:
# Load traces and experiment metadata:

for path in tqdm(path_list[21:]):
    traces = fl.load(path / "data_from_suite2p_unfiltered.h5", "/traces").T
    coords = fl.load(path / "data_from_suite2p_unfiltered.h5", "/coords")
    mask = fl.load(path / "anatomy.mask", "/mask")
    int_c = np.round(coords).astype(np.int)
    in_tectum = np.array([mask[c[0], c[2], c[1]] for c in int_c]).astype(np.bool)

    exp = EmbeddedExperiment(path)
    stim_df = stimulus_df_from_exp(exp)

    # detrend the traces
    for i in range(traces.shape[1]):
        traces[:, i] = detrend(traces[:, i])
    traces = (traces - np.nanmean(traces, 0)) / np.nanstd(traces, 0)

    # Read original frequency:
    fs = int(exp["imaging"]["microscope_config"]["lightsheet"]["scanning"]["z"]["frequency"])
    samp_n = traces.shape[0]
    t_orig = np.arange(traces.shape[0]) / fs


    # Crop around stimuli:
    n_cells = traces.shape[1]
    PRE_INT_S = 2
    POST_INT_S = 5
    cropped = crop(traces, stim_df["t"]*fs, 
                         pre_int=int(PRE_INT_S*fs), post_int=int(POST_INT_S*fs))
    cropped = cropped - cropped[:int(PRE_INT_S*fs), :, :].mean(0)

    # Find unique positions in the stimulus:
    pos = sorted(stim_df.loc[:, "pos_start"].unique())

    # Loop over positions and compute reliability scores:
    rel_scores = np.zeros((len(pos), n_cells))
    amp_scores = np.zeros((len(pos), n_cells))

    for i, p in list(enumerate(pos)):
        resps_idxs = stim_df[stim_df["pos_start"] == p].index

        rel_scores[i] = reliability(cropped[:, resps_idxs, :])
        amp_scores[i] = (cropped[8:18, resps_idxs, :].mean(0) - cropped[0:6, resps_idxs, :].mean(0)).mean(0)
    
    fl.save(path / "cell_resps.h5", dict(rel_scores=rel_scores, 
                                         in_tectum=in_tectum,
                                         amp_scores=amp_scores))

In [ ]:
all_dfs = []

for path in tqdm(path_list):
    exp = EmbeddedExperiment(path)
    gen = exp["general"]["animal"]["genotype"]
    fid = path.name
    
    cells = pd.DataFrame(fl.load(path / "cell_resps.h5", "/rel_scores").T)
    cells["in_tectum"] = fl.load(path / "cell_resps.h5", "/in_tectum")
    coords = fl.load(path / "data_from_suite2p_unfiltered.h5", "/coords")
    cells["z"] = coords[:, 0]
    cells["x"] = coords[:, 1]
    cells["y"] = coords[:, 2]
    cells["cid"] = [f"{path.name}_c{i}" for i in range(len(cells))]
    cells["gen"] = gen
    cells["fid"] = fid
    
    all_dfs.append(cells)

all_cells = pd.concat(all_dfs, axis=0)
    
all_cells = all_cells.reset_index()

In [ ]:
fl.save(master_path / "new_pooled.h5", all_cells)